In [47]:
from typing import NamedTuple, Tuple
from numpy.typing import NDArray, ArrayLike
import numpy as np
import tensorflow as tf
import sklearn.datasets as skd
import sklearn.metrics as skm
from sklearn.model_selection import train_test_split
from tqdm.auto import trange

seed = round(np.pi**15 + np.e * 21)
tf.random.set_seed(seed)
rng = np.random.default_rng(seed)
batch_size = 200
num_epochs= 25

In [48]:
def evaluate(Y_test: NDArray, preds: ArrayLike):
    print(f"R2 score: {skm.r2_score(Y_test, preds)}")
    print(f"MAE: {skm.mean_absolute_error(Y_test, preds)}")

In [49]:
X, Y = skd.fetch_california_housing(return_X_y=True)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=seed)

In [63]:
def create_model(input_shape: Tuple[int]):
    inputs = tf.keras.layers.Input(input_shape)
    x = inputs
    x = tf.keras.layers.Dense(100, activation="sigmoid")(x)
    x = tf.keras.layers.Dense(1, activation="relu")(x)  # All labels are positive
    model = tf.keras.Model(inputs=inputs, outputs=x)
    model.compile(optimizer=tf.keras.optimizers.Lion(0.0001), loss="mean_absolute_error")
    return model

model = create_model(X_train[0].shape)
model.fit(X_train, Y_train, batch_size=batch_size, epochs=50, verbose=0)
evaluate(Y_test, model.predict(X_test, verbose=0))

R2 score: 0.5767536023050737
MAE: 0.5342495543977387


In [71]:
num_clients = 10

class ClientData(NamedTuple):
    X: NDArray
    Y: NDArray


X_train_alloc = X_train.copy()
Y_train_alloc = Y_train.copy()

all_client_data = []
for c in range(num_clients - 1):
    Y_mean = rng.choice(Y_train_alloc)
    num_samples = len(Y_train) // num_clients
    idx = np.argpartition(np.abs(Y_train_alloc - Y_mean), num_samples)[:num_samples]
    client_data = ClientData(X_train_alloc[idx].copy(), Y_train_alloc[idx].copy())
    X_train_alloc, Y_train_alloc = np.delete(X_train_alloc, idx, axis=0), np.delete(Y_train_alloc, idx, axis=0)
    all_client_data.append(client_data)

all_client_data.append(ClientData(X_train_alloc.copy(), Y_train_alloc.copy()))

In [52]:
def fedavg(client_parameters, client_samples):
    agg_parameters = []
    num_layers = len(client_parameters[0])
    for i in range(num_layers):
        agg_parameters.append(np.average([cp[i] for cp in client_parameters], weights=client_samples, axis=0))
    return agg_parameters

In [72]:
num_rounds = 250

global_model = create_model(X_train[0].shape)
client_models = [create_model(X_train[0].shape) for _ in range(num_clients)]
num_client_samples = [len(cd.Y) for cd in all_client_data]

for r in (pbar := trange(num_rounds)):
    losses = []
    for client_model, client_data in zip(client_models, all_client_data):
        client_model.set_weights(global_model.get_weights())
        history = client_model.fit(client_data.X, client_data.Y, batch_size=batch_size, verbose=0)
        losses.append(history.history['loss'][0])
    global_model.set_weights(fedavg([cm.get_weights() for cm in client_models], num_client_samples))
    pbar.set_postfix_str(f"Loss: {np.average(losses, weights=num_client_samples):.3g}")


evaluate(Y_test, global_model.predict(X_test, verbose=0))

100%|██████████| 250/250 [01:29<00:00,  2.80it/s, Loss: 0.865]

R2 score: -0.08201055973888915
MAE: 0.883749073947936


: 